### Main code

In [1]:
from typing import List
import numpy as np


class InputStream:
    def __init__(self, data):
        self.data = data
        self.i = 0

    def read(self, size):
        out = self.data[self.i:self.i + size]
        self.i += size
        return int(out, 2)


def access_bit(data, num):
    """ from bytes array to bits by num position"""
    base = int(num // 8)
    shift = 7 - int(num % 8)
    return (data[base] & (1 << shift)) >> shift


def bytes2bit(data):
    """ get bit string from bytes data"""
    return ''.join([str(access_bit(data, i)) for i in range(len(data) * 8)])


def rle_to_mask(rle: List[int], height: int, width: int) -> np.array:
    """
    Converts rle to image mask
    Args:
        rle: your long rle
        height: original_height
        width: original_width

    Returns: np.array
    """

    rle_input = InputStream(bytes2bit(rle))

    num = rle_input.read(32)
    word_size = rle_input.read(5) + 1
    rle_sizes = [rle_input.read(4) + 1 for _ in range(4)]
    # print('RLE params:', num, 'values,', word_size, 'word_size,', rle_sizes, 'rle_sizes')

    i = 0
    out = np.zeros(num, dtype=np.uint8)
    while i < num:
        x = rle_input.read(1)
        j = i + 1 + rle_input.read(rle_sizes[rle_input.read(2)])
        if x:
            val = rle_input.read(word_size)
            out[i:j] = val
            i = j
        else:
            while i < j:
                val = rle_input.read(word_size)
                out[i] = val
                i += 1

    image = np.reshape(out, [height, width, 4])[:, :, 3]
    return image

In [2]:
# from PIL import Image

# image = rle_to_mask(
#     rle_id,  # result['value']['rle']
#     original_height,  # result['original_height']
#     original_width # result['original_width']
# )

# print(image.shape)  # (original_height, original_width)

# Image.fromarray(image).show()

In [3]:
def cv2_rect_to_yolo(x1,x2,y1,y2, w_size, h_size):
    w=(x2-x1)/w_size
    h=(y2-y1)/h_size
    x = x1/w_size + w/2
    y = y1/h_size + h/2

    return x,y,w,h


def write_yolo(list_yolo_labels:List[List[int]], 
               path:str):
    """
    list_yolo_labels =[ [clas, x, y, w, h], [clas, x, y, w, h] ] = List[  List[int, int, int, int, int]  ]
    """
    str_data = ""
    for i in list_yolo_labels: # clas, x, y, w, h
        i=" ".join(map(str, i))
        str_data+=f"\n{i}"

    str_data=str_data.strip()+"\n"
    with open(path, "w") as text_file:
        text_file.write(str_data)
    print("file writed")

In [4]:
def mask2yolo(arr):
    
    # print(f"{arr.shape}")
    y_len, x_len = arr.shape

    # find upper bound = Y1
    for r in range(y_len):
        if arr[r:r+1,:][0].sum()!=0:
            upper_bound = r # y1
            break


    # find down bound = Y2
    for r in reversed(range(y_len)):
        # print(f"r={r}")
        # print(f"arr[r:r+1,:] {arr[r:r+1,:]}")
        if arr[r:r+1,:][0].sum()!=0:
            down_bound = r+1 # y2
            break

    # find left bound = X1
    for r in range(x_len):
        # print(f"r={r}")
        # print(f"arr[:,r:r+1].sum() {arr[:,r:r+1].sum()}")
        if arr[:,r:r+1].sum()!=0:
            left_bound = r # x1
            break


    # find right bound = X2
    for r in reversed(range(x_len)):
        if arr[:,r:r+1].sum()!=0:
            right_bound = r+1 # x2
            break
    x1x2y1y2 = (left_bound, right_bound, upper_bound, down_bound)
    return x1x2y1y2


In [5]:
from PIL import Image

from IPython.display import display

class LSbrush_2_yolo:
    def __init__(self, path_yolo_out, classes):
        self.classes = classes# {"leaf" :0, "tomato":1, "stem":2}
        self.path_yolo_out = path_yolo_out
        
    def run(self, item_label):
        original_width = item_label["annotations"][0]["result"][0]["original_width"]
        original_height = item_label["annotations"][0]["result"][0]["original_height"]

        # Base name
        image_name = item_label['data']['image']
        image_name = os.path.basename(image_name)
        base_name = image_name[:image_name.rfind(".")]
        # print(f"image_name {image_name}")

        # np_array_base = np.zeros([original_height, original_width], dtype=np.uint8)
        list_yolo_labels=[]
        
        for result in item_label["annotations"][0]["result"]:
            rle = result["value"]["rle"]
            label = result["value"]["brushlabels"][0]
            cl_label = self.classes[label]

            image_np = rle_to_mask( rle,                               # result['value']['rle']
                                    original_height,                   # result['original_height']
                                    original_width).astype(np.uint8)   # result['original_width']  

            if image_np.max()==0:
                continue
            image_np[image_np != 0] = 254 # cl_label / value_label

            img_seg=Image.fromarray(image_np)

            # # Get value for cropp
            x1,x2,y1,y2 = mask2yolo(image_np)

            # Converting abolute rectangle point [x1,x2,y1,y2] to => relative yolo [x, y, w, h]
            x,y,w,h = cv2_rect_to_yolo(x1,x2,y1,y2, original_width, original_height)
            # Writing results to yolo label file
            list_yolo_labels.append([cl_label, x, y, w, h])

        path_yolo_label = os.path.join(self.path_yolo_out, f"{base_name}.txt")
        write_yolo(list_yolo_labels, path_yolo_label)
                 

In [6]:
import json
import os
from tqdm import tqdm

path_labeled = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Olga/project-21-at-2024-01-29-09-05-9fc962dd.json"
path_yolo_out = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Olga/yolo"
classes = {"leaf" :0, "tomato":1, "stem":2}

with open(path_labeled) as user_file:
    parsed_json = json.load(user_file)

for item_label in tqdm(parsed_json):
    c=LSbrush_2_yolo(path_yolo_out, classes)
    image_np=c.run(item_label)


 11%|█████████████████▎                                                                                                                                          | 1/9 [00:12<01:38, 12.32s/it]

file writed


 22%|██████████████████████████████████▋                                                                                                                         | 2/9 [00:18<00:59,  8.56s/it]

file writed


 33%|████████████████████████████████████████████████████                                                                                                        | 3/9 [00:25<00:49,  8.17s/it]

...

file writed


 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 8/9 [01:05<00:07,  7.95s/it]

file writed


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:08<00:00,  7.65s/it]

file writed


### Train test scripts

In [ ]:
image_np.max()

In [ ]:
arr = np.zeros([12,6])
arr[4:7,3:5]=1
arr

In [ ]:
# arr[11][-1]=100
# arr[11][-2]=99
# arr[10][-2]=98
# arr[0][0]=50


arr

In [ ]:
arr

In [ ]:


# mask2yolo(arr)    

In [ ]:
x1,x2,y1,y2 =mask2yolo(arr) 
print(f"x1,x2,y1,y2 {x1,x2,y1,y2}")
arr[y1:y2, x1:x2]

In [ ]:
r=1
arr[:,r:r+1][0]

In [ ]:
arr[:,4:11]

In [ ]:
image_np[8:9,:]

In [ ]:
arr

In [ ]:
a=[0,1,2,3,4]

" ".join(map(str, a))

In [ ]:
a = "image_name 148.png"

a[:a.rfind(".")]